---
title: "Accelerate, Three Powerful Sublibraries for PyTorch"
author: "Zachary Mueller"
format: 
    revealjs:
        theme: moon
        fig-format: png
---

## Who am I?

- Zachary Mueller
- Deep Learning Software Engineer at 🤗
- API design geek

## What is 🤗 Accelerate?

```{mermaid}
%%| fig-height: 6
graph LR
    A{"🤗 Accelerate#32;"}
    A --> B["Launching<br>Interface#32;"]
    A --> C["Training Library#32;"]
    A --> D["Big Model<br>Inference#32;"]
```

# A Launching Interface

Can't I just use `python do_the_thing.py`?

## A Launching Interface

Launching scripts in different environments is complicated:

- ```bash 
python script.py
```

- ```bash 
torchrun --nnodes=1 --nproc_per_node=2 script.py
```

- ```bash 
deepspeed --num_gpus=2 script.py
```

And more!

## A Launching Interface

But it doesn't have to be:

```bash
accelerate launch script.py
```

A single command to launch with `DeepSpeed`, Fully Sharded Data Parallelism, across single and multi CPUs and GPUs, and to train on TPUs[^1] too! 

[^1]: Without needing to modify your code and create a `_mp_fn`

## A Launching Interface

Generate a device-specific configuration through `accelerate config`

![](CLI.gif)

## A Launching Interface

Or don't. `accelerate config` doesn't *have* to be done!

```bash
torchrun --nnodes=1 --nproc_per_node=2 script.py
accelerate launch --multi_gpu --nproc_per_node=2 script.py
```

A quick default configuration can be made too:

```bash 
accelerate config default
```

## A Launching Interface

With the `notebook_launcher` it's also possible to launch code directly from your Jupyter environment too!

```python
from accelerate import notebook_launcher
notebook_launcher(
    training_loop_function, 
    args, 
    num_processes=2
)
```

```python
Launching training on 2 GPUs.
epoch 0: 88.12
epoch 1: 91.73
epoch 2: 92.58
epoch 3: 93.90
epoch 4: 94.71
```

# A Training Library

Okay, will `accelerate launch` make `do_the_thing.py` use all my GPUs magically?

## A Training Library

- Just showed that its possible using `accelerate launch` to *launch* a python script in various distributed environments
- This does *not* mean that the script will just "use" that code and still run on the new compute efficiently.
- Training on different computes often means *many* lines of code changed for each specific compute.
- 🤗 `accelerate` solves this by ensuring the same code can be ran on a CPU or GPU, multiples, and on TPUs!

## A Training Library


```{.python}
for batch in dataloader:
    optimizer.zero_grad()
    inputs, targets = batch
    inputs = inputs.to(device)
    targets = targets.to(device)
    outputs = model(inputs)
    loss = loss_function(outputs, targets)
    optimizer.step()
    scheduler.step()
```

## A Training Library {.smaller}

:::: {.columns}
::: {.column width="43%"}
<br><br><br>
```{.python code-line-numbers="5-6,9"}
# For alignment purposes
for batch in dataloader:
    optimizer.zero_grad()
    inputs, targets = batch
    inputs = inputs.to(device)
    targets = targets.to(device)
    outputs = model(inputs)
    loss = loss_function(outputs, targets)
    optimizer.step()
    scheduler.step()
```
:::
::: {.column width="57%"}
```{.python code-line-numbers="1-7,12-13,16"}
from accelerate import Accelerator
accelerator = Accelerator()
dataloader, model, optimizer scheduler = (
    accelerator.prepare(
        dataloader, model, optimizer, scheduler
    )
)

for batch in dataloader:
    optimizer.zero_grad()
    inputs, targets = batch
    # inputs = inputs.to(device)
    # targets = targets.to(device)
    outputs = model(inputs)
    loss = loss_function(outputs, targets)
    accelerator.step(loss) # optimizer.step()
    scheduler.step()
```
:::

::::

## A Training Library

What all happened in `Accelerator.prepare`?

::: {.incremental}
1. `Accelerator` looked at the configuration
2. The `training_dataloader` was converted into one that can dispatch each batch onto a seperate GPU
3. The `model` was wrapped with the appropriate DDP wrapper from either `torch.distributed` or `torch_xla`
4. The `optimizer` and `scheduler` were both converted into an `AcceleratedOptimizer` and `AcceleratedScheduler` which knows how to handle any distributed scenario
:::